In [30]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import scipy.io
from scipy.signal import resample


In [31]:
labels_df = pd.read_csv('REFERENCE-v3.csv', header=None)
labels_df.columns = ['filename', 'label']

# Fonction pour charger un fichier ECG au format .mat
def load_ecg_file(filename):
    mat = scipy.io.loadmat(f'training2017/{filename}.mat')
    signal = mat['val'][0]
    return signal

In [32]:
example_signal = load_ecg_file(labels_df['filename'].iloc[0])
print(example_signal[:10])
print(len(example_signal))

[-127 -162 -197 -229 -245 -254 -261 -265 -268 -268]
9000


In [33]:
def preprocess_signal(signal, target_length=300):
    return resample(signal, target_length)

# Prétraitement d'un exemple de signal
preprocessed_signal = preprocess_signal(example_signal)
print(preprocessed_signal)


[ -86.09377424 -285.3294793  -185.02034692 -269.329934    104.63599691
 -199.91048176  -43.23759037   44.44623751   35.64991909  126.02866827
  149.96457007  338.9555403   260.51561158   88.43924386   11.5876541
 -483.0757818  -562.87428834 -353.16632079  106.16229898    9.83256683
  -74.91087094  177.18789248  146.56026921   55.6490158   167.78126219
   19.32176288  234.6480642   104.58702469  -47.36326762   66.63623167
   -6.70483045  -11.37414113   24.66255914  -26.14973227  150.78780592
  105.81242612 -113.62434463   55.89158899  -25.93342562  -27.87917845
   29.85168779  -14.05313866  237.0854506     3.53035779  -33.87587388
   42.34202002  -13.38152309  -20.75539866   33.57937055  -13.44632366
  244.58668211  -15.81143209  -37.56283803   40.32654927  -28.87352829
  -10.87580302   37.35561505  -12.26090752  213.72858202   41.31701895
  -78.17941326   60.43810715  -57.41309765  -17.52953383   15.0969719
   -2.28361861  212.09862029   89.64702111  -68.94717011   49.59725565
 -110.02

In [34]:
from torch.utils.data import Dataset, DataLoader

class ECGDataset(Dataset):
    def __init__(self, labels_df):
        self.labels_df = labels_df

    def __len__(self):
        return len(self.labels_df)

    def __getitem__(self, idx):
        filename = self.labels_df.iloc[idx, 0]
        label = self.labels_df.iloc[idx, 1]
        signal = preprocess_signal(load_ecg_file(filename))
        return torch.tensor(signal, dtype=torch.float32), label

# Créer le Dataset
dataset = ECGDataset(labels_df)
dataloader = DataLoader(dataset, batch_size=32, shuffle=True)


In [35]:
class ECG_RNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, num_layers=1):
        super(ECG_RNN, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        # RNN layer
        self.rnn = nn.RNN(input_size, hidden_size, num_layers, batch_first=True)
        # Fully connected layer for classification
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        # Initialize hidden state with zeros
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)
        # Forward propagate the RNN
        out, _ = self.rnn(x, h0)
        # We only need the output of the last time step
        out = self.fc(out[:, -1, :])
        return out

# Paramètres du modèle
input_size = 1  # 1D signal
hidden_size = 64
output_size = 4  # N, AF, Other, Noisy
model = ECG_RNN(input_size, hidden_size, output_size)


In [36]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)


In [37]:
num_epochs = 10

for epoch in range(num_epochs):
    for signals, labels in dataloader:
        signals = signals.unsqueeze(-1)  # Ajouter une dimension pour l'entrée RNN (batch_size, sequence_length, input_size)
        labels = torch.tensor([{'N': 0, 'A': 1, 'O': 2, '~': 3}[l] for l in labels])
        
        # Clear gradients
        optimizer.zero_grad()
        
        # Forward pass
        outputs = model(signals)
        loss = criterion(outputs, labels)
        
        # Backward pass and optimization
        loss.backward()
        optimizer.step()

    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')


Epoch [1/10], Loss: 0.8491
Epoch [2/10], Loss: 0.8386
Epoch [3/10], Loss: 0.9993
Epoch [4/10], Loss: 0.8614
Epoch [5/10], Loss: 0.9071
Epoch [6/10], Loss: 0.7822
Epoch [7/10], Loss: 1.1803
Epoch [8/10], Loss: 1.0016
Epoch [9/10], Loss: 0.7980
Epoch [10/10], Loss: 1.1349


In [20]:
label_mapping = {'N': 0, 'A': 1, 'O': 2, '~': 3}

# Modification dans la boucle d'évaluation
model.eval()
y_true = []
y_pred = []

with torch.no_grad():
    for signals, labels in dataloader:
        signals = signals.unsqueeze(-1)
        outputs = model(signals)
        _, predicted = torch.max(outputs.data, 1)
        
        # Convertir les labels (y_true) en nombres
        labels = torch.tensor([label_mapping[l] for l in labels])

        y_true.extend(labels)
        y_pred.extend(predicted)

# Générer le rapport de classification
print(classification_report(y_true, y_pred, target_names=['N', 'A', 'O', '~']))


              precision    recall  f1-score   support

           N       0.60      1.00      0.75      5076
           A       0.00      0.00      0.00       758
           O       0.00      0.00      0.00      2415
           ~       0.27      0.04      0.07       279

    accuracy                           0.59      8528
   macro avg       0.22      0.26      0.20      8528
weighted avg       0.36      0.59      0.45      8528



c:\Users\Albus\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\Albus\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\Albus\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
